In [1]:
import os
from tensorflow import keras
from keras import *
from keras.layers import *
from keras.models import *
from keras.preprocessing.image import ImageDataGenerator
import random
import shutil
import keras
import keras.utils
from keras.utils.np_utils import to_categorical

In [ ]:
# Подготовка данных

In [2]:
# получение данных
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import os
import pprint
pp = pprint.PrettyPrinter(indent = 4)

import joblib
from skimage.io import imread
from skimage.transform  import resize

def resize_all(src, pklname, include, width = 150, height=None):
    height = height if height is not None else width
    
    data = dict()
    data['description'] = 'resized ({0}x{1})animal images in rgb'.format(int(width), int(height))
    data['label'] = []
    data['filename'] = []
    data['data'] = []
    
    pklname = f"{pklname}_{width}x{height}px.pkl"
    
    for subdir in os.listdir(src):
        if subdir in include:
            print(subdir)
            current_path = os.path.join(src,subdir)
            
            for file in os.listdir(current_path):
                if file[-3:] in {'jpg', 'png'}:
                    im = imread(os.path.join(current_path, file))
                    im = resize(im, (width, height))
                    if im.shape == (150,150,3):
                        data['label'].append(subdir[:-4])
                        data['filename'].append(file)
                        data['data'].append(im)
                    
        joblib.dump(data, pklname)

In [3]:
data_path = "D:/ashikh/work/CV/AnimalFaces_choice/train"
data_path_test = "D:/ashikh/work/CV/AnimalFaces_choice/test"
base_name = 'animal_faces'
base_name_test = 'animal_faces_test'
width = 150
include = {'BearHead', 'CatHead', 'ChickenHead', 
           'CowHead', 'DeerHead', 'DogHead', 'DuckHead',
           'EagleHead', 'ElephantHead', 'HumanHead', 'LionHead', 'MonkeyHead',
           'MouseHead', 'PandaHead', 'PigeonHead', 'PigHead', 'RabbitHead',
           'SheepHead', 'TigerHead', 'WolfHead'}

# include = {'BearHead', 'CatHead', 'ChickenHead', 
#            'CowHead', 'DeerHead', 'DogHead', 'DuckHead'}

resize_all(src=data_path, pklname=base_name, width=width, include=include)
resize_all(src=data_path_test, pklname=base_name_test, width=width, include=include)

data = joblib.load(f'{base_name}_{width}x{width}px.pkl')
data_test = joblib.load(f'{base_name_test}_{width}x{width}px.pkl')



BearHead
CatHead
ChickenHead
CowHead
DeerHead
DogHead
DuckHead
EagleHead
ElephantHead
HumanHead
LionHead
MonkeyHead
MouseHead
PandaHead
PigeonHead
PigHead
RabbitHead
SheepHead
TigerHead
WolfHead
BearHead
CatHead
ChickenHead
CowHead
DeerHead
DogHead
DuckHead
EagleHead
ElephantHead
HumanHead
LionHead
MonkeyHead
MouseHead
PandaHead
PigeonHead
PigHead
RabbitHead
SheepHead
TigerHead
WolfHead


In [4]:
def y_preproc(y_train, y_test):
    temp = []
    for i in y_train:
        for j in range(len(class_name)):
            if i == class_name[j]:
                temp.append(j)
    y_train = temp
    temp = []
    for i in y_test:
        for j in range(len(class_name)):
            if i == class_name[j]:
                temp.append(j)
    y_test = temp
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return y_train, y_test

In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import defaultdict

X_train = np.array(data['data'])
y_train = np.array(data['label'])
X_test = np.array(data_test['data'])
y_test = np.array(data_test['label'])


class_name = ['Bear', 'Cat', 'Chicken', 
           'Cow', 'Deer', 'Dog', 'Duck',
           'Eagle', 'Elephant', 'Human', 'Lion', 'Monkey',
           'Mouse', 'Panda', 'Pigeon', 'Pig', 'Rabbit',
           'Sheep', 'Tiger', 'Wolf']

# class_name = ['Bear', 'Cat', 'Chicken', 
#            'Cow', 'Deer', 'Dog', 'Duck']

y_train, y_test = y_preproc(y_train, y_test)

In [ ]:
# Генерация модели

In [43]:
img_width, img_height = 150, 150

input_shape = (150, 150, 3)
epochs = 20
batch_size = 20


In [47]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(784))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(20))
model.add(Activation('sigmoid'))

In [48]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [49]:
# Обучение 

model.fit(
        X_train,
        y_train,
        batch_size = batch_size,
        epochs = epochs,
        validation_data = (X_test, y_test),
)

Epoch 1/20
97/97 [==============================] - 21s 214ms/step - loss: 2.5249 - accuracy: 0.2518 - val_loss: 1.8191 - val_accuracy: 0.4455
Epoch 2/20
97/97 [==============================] - 21s 216ms/step - loss: 1.3689 - accuracy: 0.5931 - val_loss: 1.2884 - val_accuracy: 0.6227
Epoch 3/20
97/97 [==============================] - 21s 216ms/step - loss: 0.7242 - accuracy: 0.7823 - val_loss: 1.3182 - val_accuracy: 0.6455
Epoch 4/20
97/97 [==============================] - 21s 215ms/step - loss: 0.3455 - accuracy: 0.8961 - val_loss: 1.3927 - val_accuracy: 0.6205
Epoch 5/20
97/97 [==============================] - 21s 215ms/step - loss: 0.1784 - accuracy: 0.9473 - val_loss: 1.3373 - val_accuracy: 0.6773
Epoch 6/20
97/97 [==============================] - 21s 216ms/step - loss: 0.0776 - accuracy: 0.9772 - val_loss: 1.4043 - val_accuracy: 0.6955
Epoch 7/20
97/97 [==============================] - 21s 216ms/step - loss: 0.1040 - accuracy: 0.9741 - val_loss: 1.5653 - val_accuracy: 0.6841

In [51]:
model.save("D:/ashikh/work/CV/models/animal_classifer_v1")

INFO:tensorflow:Assets written to: D:/ashikh/work/CV/models/animal_classifer_v1\assets


In [ ]:
# Тест модели на примере

In [ ]:
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [ ]:
model = keras.models.load_model("D:/ashikh/work/CV/English/mymodel4")

In [ ]:
img_path = "D:/ashikh/work/CV/duckside0017.jpg"


In [ ]:
im = imread(img_path)
im = resize(im, (150, 150))
x = np.expand_dims(im, axis=0)
p = model.predict(x)

In [ ]:
maxx = p[0][0]
maxx_i = 0
for i in range(1, 7):
    if p[0][i] > maxx:
        maxx = p[0][i]
        maxx_i = i
iden = ['Bear', 'Cat', 'Chicken', 
           'Cow', 'Deer', 'Dog', 'Duck']


fig, axes = plt.subplots(1)
# fig.set_size_inches(15,4)
# fig.tight_layout()

idx = iden[maxx_i]
axes.imshow(im)
axes.axis('off')
# axes.set_title(idx)
    
print(iden[maxx_i])